# "Useful tools for transforming the input features"
> "Incredibly usefull tool to analyze and preprocess data for a deep neural network"


- toc: true
- branch: master
- badges: false
- comments: true
- categories: [data, eda, pandas]
- image: images/wine_dataset/iStock-615737086-768x512.jpg
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [ ]:
#hide
# Standard Library Imports

%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install fastai --upgrade --q
from pathlib import Path
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
import matplotlib.pyplot as plt

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
import numpy as np
from scipy.stats import kurtosis, skew
from scipy import stats
import math
import warnings
warnings.filterwarnings("error")

from google.colab import drive
mnt=drive.mount('/content/gdrive', force_remount=True)



     |████████████████████████████████| 194kB 6.0MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
Mounted at /content/gdrive


# I will explore the transformations using the [wine data set](https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009) from Kaggle.

In [ ]:
#hide
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'redwine'
csv_path = (base_dir+'/winequality-red.csv')
df=pd.read_csv(csv_path)

# The following numerical transformations will be evaluated on the dataset:


*   Log 
*   Square Root
*   Box-Cox
*   Yeo-Yohnson
*   Quantile
*   Clipping

